In [55]:
!pip install transformers
!pip install ipywidgets
!pip install -U tensorflow-text==2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [57]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

import tensorflow_hub as hub
from tensorflow.keras.models import Model
import tensorflow_text
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [12]:
# %cd "D:\Study\Chatbot\DL_bot"
%cd "/content/drive/My Drive/Study/Chatbot/DL_bot"

/content/drive/My Drive/Study/Chatbot/DL_bot


In [13]:
reviews_df = pd.read_csv('amazon_vfl_reviews.csv')

In [14]:
reviews_df.shape

(2782, 5)

In [15]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782 entries, 0 to 2781
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   asin    2782 non-null   object
 1   name    2782 non-null   object
 2   date    2782 non-null   object
 3   rating  2782 non-null   int64 
 4   review  2778 non-null   object
dtypes: int64(1), object(4)
memory usage: 108.8+ KB


In [16]:
reviews_df.drop(['asin','name','date'],axis=1,inplace=True)

In [17]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782 entries, 0 to 2781
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  2782 non-null   int64 
 1   review  2778 non-null   object
dtypes: int64(1), object(1)
memory usage: 43.6+ KB


In [18]:
reviews_df.head(2)

,rating,review
0,1,I bought this hair oil after viewing so many g...
1,5,Used This Mama Earth Newly Launched Onion Oil ...


In [19]:
reviews_df.rating.value_counts()

5    1444
1     546
4     464
3     198
2     130
Name: rating, dtype: int64

In [20]:
reviews_df['sent_rating'] = reviews_df.rating.apply(lambda x: 1 if x>=4 else 0)

In [21]:
reviews_df.sent_rating.value_counts()

1    1908
0     874
Name: sent_rating, dtype: int64

In [22]:
reviews_df.drop('rating',axis=1,inplace=True)

In [103]:
import re
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

def clean_text(review_text):
  text = str(review_text).lower()
  text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
  text = " ".join([word for word in text.split() if word not in (stop)])
  text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
  return text






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [104]:
reviews_df['review'] = reviews_df.review.apply(clean_text)

In [105]:
X_train, X_test, y_train, y_test = train_test_split(reviews_df['review'],reviews_df['sent_rating'],test_size=0.3,random_state=1234,stratify=reviews_df['sent_rating'])

In [106]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1947,), (1947,), (835,), (835,))

In [107]:
tf.keras.backend.clear_session()

In [108]:
#BERT takes 3 inputs - input word ids, input mask for padding, segment ids - for sentences

def create_model():
   
        
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

        #bert layer 
    bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

    preprocessed_text = bert_preprocess(text_input)
    output = bert_layer(preprocessed_text)

    output = tf.keras.layers.Dense(1,"sigmoid")(output['pooled_output'])
    #Bert model
    #We are using only pooled output not sequence out. 
    #If you want to know about those, please read https://www.kaggle.com/questions-and-answers/86510
    model = Model(inputs=text_input, outputs=output)

        
    return model

In [109]:
model = create_model()

In [110]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'default': (None, 7 109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [111]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [113]:
model.fit(X_train, y_train, validation_data = (X_test,y_test),epochs=10)

Epoch 1/10
61/61 [==============================] - 20s 328ms/step - loss: 0.5433 - accuracy: 0.7242 - precision: 0.7244 - recall: 0.9648 - val_loss: 0.5462 - val_accuracy: 0.7341 - val_precision: 0.7455 - val_recall: 0.9302
Epoch 2/10
61/61 [==============================] - 18s 302ms/step - loss: 0.5332 - accuracy: 0.7304 - precision: 0.7293 - recall: 0.9648 - val_loss: 0.5378 - val_accuracy: 0.7114 - val_precision: 0.7134 - val_recall: 0.9686
Epoch 3/10
61/61 [==============================] - 18s 302ms/step - loss: 0.5313 - accuracy: 0.7375 - precision: 0.7424 - recall: 0.9453 - val_loss: 0.5310 - val_accuracy: 0.7150 - val_precision: 0.7139 - val_recall: 0.9756
Epoch 4/10
61/61 [==============================] - 18s 302ms/step - loss: 0.5152 - accuracy: 0.7406 - precision: 0.7399 - recall: 0.9588 - val_loss: 0.5223 - val_accuracy: 0.7521 - val_precision: 0.7577 - val_recall: 0.9389
Epoch 5/10
61/61 [==============================] - 18s 302ms/step - loss: 0.5106 - accuracy: 0.7447

In [114]:
y_pred = model.predict(X_test[:10])

In [121]:
y_pred[y_pred <= 0.5] = 0.
y_pred[y_pred > 0.5] = 1.

In [124]:
len(y_pred)

9

In [122]:
from sklearn.metrics import confusion_matrix, classification_report


In [125]:
confusion_matrix(y_test[:9],y_pred)

array([[0, 1],
       [0, 8]])

In [127]:

print(classification_report(y_test[:9], y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.89      1.00      0.94         8

    accuracy                           0.89         9
   macro avg       0.44      0.50      0.47         9
weighted avg       0.79      0.89      0.84         9



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [128]:
model.save('sentiment_analyzer')

In [129]:
from tensorflow.keras import models

In [131]:
mymodel = models.load_model("sentiment_analyzer")

In [136]:
mymodel.predict(['This is my review. Not a great product'])

array([[0.6081653]], dtype=float32)

In [137]:
mymodel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________